## Introduction
All data are time-series data; the maximum granularity of time is quarterly, the smaller granularity is monthly, and occasionally there are several points in an individual month; we use the average value as the current month's data so that the minimum granularity can be considered as monthly.

The data will be uniformly converted into quarterly time scales:

- When data are transformed, quarterly data for non-rate of change will simply be treated as an arithmetic average of monthly data, such as the unemployment rate data.
  $$unemployment\_rate\_Q1 = \frac{unemployment\_rate\_Jan + unemployment\_rate\_Feb + unemployment\_rate\_Mar} {3}$$
- When the data are converted, the quarterly data for the rate of change will simply be treated as the product of the months, as is the case with CPI data.
  $$CPI\_Q1 = ((1 + CPI\_Jan) \times (1 + CPI\_Feb) \times (1 + CPI\_Mar)) - 1$$

## Data
Each data contains different start time, we first find a common time start point of each data, this time point is **January 1999** is the start time point of OCR data, and the end of duration is **September, 2022**. This notebook will wrangling the following data:
- HPI
- CPI
- OCR
- Exchange rate
- Unemployment rate

In [1]:
# Normalization for each time formats
# install.packages("timeDate")
library(timeDate)
library(tidyverse)
library(visdat)
library(here)
library(readxl)

# convert "1999Q1" to "1999-03"
Yq2Ym <- function(date) {
           dt = str_split(date, "Q")
           y = dt[[1]][[1]]
           q = strtoi(dt[[1]][[2]], 10)
           m = q * 3
           mm = str_pad(m, 2, side = "left", pad = 0)
           ym = paste(y, mm, sep = "-")
           ym
       }

# convert "1999-3-29" to "1999-03"
Yqd2Ym <- function(date) {
           dt = str_split(date, "-")
           y = dt[[1]][[1]]
           m = strtoi(dt[[1]][[2]], 10)
           mm = str_pad(m, 2, side = "left", pad = 0)
           ym = paste(y, mm, sep = "-")
           ym
       }

# Convert "1999Q1" to the last day of quarter
Yq2Date <- function(date) {
           ym = Yq2Ym(date)
           ymd = paste(ym, "01", sep = "-")
           dt = as.Date(timeLastDayInMonth(ymd))
           dt
       }


# Convert "1999-03-29" to the last day of month
Yqd2Date <- function(date) {
          ym = Yqd2Ym(date)
          ymd = paste(ym, "01", sep = "-")
          dt = as.Date(timeLastDayInMonth(ymd))
          dt
       }

# convert "1999-03" to "1999Q1"
Ym2Yq <- function(date) {
           dt = str_split(date, "-")
           y = dt[[1]][[1]]
           m = strtoi(dt[[1]][[2]], 10)
           q = (m - 1) %/% 3 + 1
           yq = paste(y, q, sep = "Q")
           yq
       }

-- Attaching packages --------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.3.6      v purrr   0.3.4 
v tibble  3.1.8      v dplyr   1.0.10
v tidyr   1.2.0      v stringr 1.4.1 
v readr   2.1.2      v forcats 0.5.2 
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
here() starts at /Users/zhoujinze/Documents/DataScience/ds-assessment/DATA-422/group project/data422-group-project



In [2]:
hpi <- read_csv("data/hpi.csv", col_select = c("date", "HPI"), show_col_types = FALSE)

In [3]:
q_hpi <- hpi %>%
    mutate(date.quarter = map_chr(hpi$date, Yqd2Ym), date.datetime = map(hpi$date, Yqd2Date))%>%
    filter(date.datetime > as.Date("1999-01-01")) %>%
    select(date.quarter, HPI)
    
q_hpi %>% head(10)

date.quarter,HPI
<chr>,<dbl>
1999-03,699
1999-06,703
1999-09,704
1999-12,704
2000-03,703
2000-06,696
2000-09,696
2000-12,696
2001-03,700


In [4]:
cpi <- read_csv("data/cpi_nz.csv", col_select = c("date", "CPI"), show_col_types = FALSE)

In [5]:
q_cpi <- cpi %>%
    mutate(date.quarter = map_chr(cpi$date, Yqd2Ym), date.datetime = map(cpi$date, Yqd2Date))%>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01")) %>%
    select(date.quarter, CPI)
    
q_cpi %>% head(10)

date.quarter,CPI
<chr>,<dbl>
1999-03,-0.09
1999-06,-0.36
1999-09,-0.51
1999-12,0.51
2000-03,1.49
2000-06,2.00
2000-09,2.99
2000-12,3.98
2001-03,3.06


In [6]:
# OCR - official cash rate
ocr <- read_csv("data/official_cash_rate.csv", col_select = c("date", "ocr"), show_col_types = FALSE)
ocr$date.month = map_chr(ocr$date, Yqd2Ym)
ocr$date.datetime = map(ocr$date, Yqd2Date)

In [7]:
m_ocr <- ocr %>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01")) %>%
    group_by(date.month) %>%
    summarise(ocr_month = mean(ocr))

# completing missing data for the month
month_seq = map_chr(seq(as.Date("1999-1-1"), as.Date("2022-9-1"), "month"), Yqd2Ym)

all_month_ocr <- merge(
     x = data.frame(date.month = month_seq),
     y = m_ocr,
     all.x = TRUE) %>%
    fill(ocr_month) %>% fill(ocr_month, .direction = "downup")

all_month_ocr$date.quarter = map_chr(all_month_ocr$date.month, Ym2Yq)
    
q_ocr <- all_month_ocr %>%
    group_by(date.quarter) %>%
    summarise(ocr_quarter = mean(ocr_month)) %>%
    mutate(date.quarter = map_chr(.$date.quarter, Yq2Ym))

q_ocr %>% head(10)

date.quarter,ocr_quarter
<chr>,<dbl>
1999-03,4.500000
1999-06,4.500000
1999-09,4.500000
1999-12,4.666667
2000-03,5.583333
2000-06,6.333333
2000-09,6.500000
2000-12,6.500000
2001-03,6.458333


In [8]:
# exchange rate NZD-USD
nzd_usd <- read_csv("data/nzd-usd_exchange_rate.csv", col_select = c("date", "USD exchange rate"), show_col_types = FALSE)
nzd_usd$date.month = map_chr(nzd_usd$date, Yqd2Ym)
nzd_usd$date.datetime = map(nzd_usd$date, Yqd2Date)

m_nzd_usd <- nzd_usd %>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01"))

m_nzd_usd$date.quarter = map_chr(m_nzd_usd$date.month, Ym2Yq)
# m_nzd_usd$usd_rate_m = m_nzd_usd$'USD exchange rate'
names(m_nzd_usd)[names(m_nzd_usd) == "USD exchange rate"] <- "usd_rate_m"

q_nzd_usd <- m_nzd_usd %>%
    group_by(date.quarter) %>%
    summarise(usd_rate_quarter = mean(usd_rate_m)) %>%
    mutate(date.quarter = map_chr(.$date.quarter, Yq2Ym))
q_nzd_usd %>% head(10)

date.quarter,usd_rate_quarter
<chr>,<dbl>
1999-03,0.5383333
1999-06,0.5426667
1999-09,0.5253333
1999-12,0.5120000
2000-03,0.4986667
2000-06,0.4793333
2000-09,0.4420000
2000-12,0.4093333
2001-03,0.4333333


In [9]:
# exchange rate NZD-CNY
nzd_cny <- read_csv("data/nzd-cny_exchange_rate.csv", col_select = c("date", "NZD_CNY_rate"), show_col_types = FALSE)
nzd_cny$date.month = map_chr(nzd_cny$date, Yqd2Ym)
nzd_cny$date.datetime = map(nzd_cny$date, Yqd2Date)

m_nzd_cny <- nzd_cny %>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01"))

m_nzd_cny$date.quarter = map_chr(m_nzd_cny$date.month, Ym2Yq)

q_nzd_cny <- m_nzd_cny %>%
    group_by(date.quarter) %>%
    summarise(cny_rate_quarter = mean(NZD_CNY_rate)) %>%
    mutate(date.quarter = map_chr(.$date.quarter, Yq2Ym))
q_nzd_cny %>% head(10)

date.quarter,cny_rate_quarter
<chr>,<dbl>
1999-03,4.457178
1999-06,4.496075
1999-09,4.349201
1999-12,4.235615
2000-03,4.119027
2000-06,3.961808
2000-09,3.651363
2000-12,3.394744
2001-03,3.579958


In [10]:
# unemployment rate
unemployment_rate <- read_excel("data/labour_force_status.xlsx") %>%
    select(date, "Unemployment Rate")

names(unemployment_rate)[names(unemployment_rate) == "Unemployment Rate"] <- "ump_rate"

q_ump_rate <- unemployment_rate %>%
    mutate(date.quarter = map_chr(unemployment_rate$date, Yq2Ym), date.datetime = map(unemployment_rate$date, Yq2Date))%>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01")) %>%
    select(date.quarter, ump_rate)

q_ump_rate %>% head(10)

date.quarter,ump_rate
<chr>,<chr>
1999-03,7.6
1999-06,7.4
1999-09,7.0
1999-12,6.5
2000-03,6.6
2000-06,6.4
2000-09,6.1
2000-12,5.9
2001-03,5.6


In [11]:
# tourism data
tourism <- read_csv("data/tourism_final_clean_arriavals_1921Q1-2022Q2.csv", show_col_types = FALSE)
#data_tourisms <- read_csv("data/tourism_final_clean_arriavals_1921Q1-2022Q2.csv", col_select = c("Id", "date.quarter","Actual_counts"), show_col_types = FALSE)
names(tourism)[names(tourism) == "date"] <- "date.quarter"
names(tourism)[names(tourism) == "Actual_counts"] <- "tourism_count"

q_tourism <- tourism %>% 
    mutate(date.datetime = map(tourism$date.quarter, Yqd2Date)) %>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01")) %>%
    select(date.quarter, tourism_count)

q_tourism %>% tail(5)

New names:
* `` -> `...1`


date.quarter,tourism_count
<chr>,<dbl>
2021-06,141126
2021-09,35195
2021-12,15157
2022-03,37892
2022-06,221706


In [14]:
# imports & exports data
imports <- read_csv("data/total_imports_final_clean_1988Q1-2022Q3.csv", show_col_types = FALSE)
names(imports)[names(imports) == "date"] <- "date.quarter"
names(imports)[names(imports) == "Total_Gross_Imports_in_tonnes"] <- "imports_total_gross_in_tonnes"

q_imports <- imports %>% 
    mutate(date.datetime = map(imports$date.quarter, Yqd2Date)) %>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01")) %>%
    select(date.quarter, imports_total_gross_in_tonnes)

# exports
exports <- read_csv("data/total_exports_final_clean_1988Q1-2022Q3.csv", show_col_types = FALSE)
names(exports)[names(exports) == "date"] <- "date.quarter"
names(exports)[names(exports) == "Total_Gross_Exports_in_tonnes"] <- "exports_total_gross_in_tonnes"

q_exports <- exports %>% 
    mutate(date.datetime = map(exports$date.quarter, Yqd2Date)) %>%
    filter(date.datetime > as.Date("1999-01-01") & date.datetime < as.Date("2022-10-01")) %>%
    select(date.quarter, exports_total_gross_in_tonnes)

q_imports %>% head(5)
q_exports %>% tail(5)

New names:
* `` -> `...1`
New names:
* `` -> `...1`


date.quarter,imports_total_gross_in_tonnes
<chr>,<dbl>
1999-03,3313467
1999-06,3188184
1999-09,3454552
1999-12,3441761
2000-03,3367151


date.quarter,exports_total_gross_in_tonnes
<chr>,<dbl>
2021-09,10059054
2021-12,10216018
2022-03,8925030
2022-06,10670814
2022-09,9478434


In [15]:
all_data <- q_ocr %>%
    left_join(q_cpi) %>%
    left_join(q_hpi) %>%
    left_join(q_ump_rate) %>%
    left_join(q_nzd_usd) %>%
    left_join(q_nzd_cny) %>% 
    left_join(q_tourism) %>%
    left_join(q_exports) %>%
    left_join(q_imports) %>%
    fill(ump_rate) %>% fill(tourism_count)

all_data %>% tail(10)

Joining, by = "date.quarter"
Joining, by = "date.quarter"
Joining, by = "date.quarter"
Joining, by = "date.quarter"
Joining, by = "date.quarter"
Joining, by = "date.quarter"
Joining, by = "date.quarter"
Joining, by = "date.quarter"


date.quarter,ocr_quarter,CPI,HPI,ump_rate,usd_rate_quarter,cny_rate_quarter,tourism_count,exports_total_gross_in_tonnes,imports_total_gross_in_tonnes
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020-06,0.3750000,1.45,2807.4,4.2,0.6176667,4.382507,7431,9551649,4930853
2020-09,0.2500000,1.44,2827.4,5.4,0.6620000,4.579023,13782,10577978,5737541
2020-12,0.2500000,1.44,3029.2,5.1,0.6860000,4.546989,16093,11379776,6346453
2021-03,0.2500000,1.52,3230.7,4.8,0.7190000,4.653247,15384,10412001,5916959
2021-06,0.2500000,3.34,3447.5,4.2,0.7153333,4.617931,141126,11752629,6041585
2021-09,0.2500000,4.93,3613.5,3.5,0.7003333,4.534847,35195,10059054,6713726
2021-12,0.5833333,5.95,3865.2,3.4,0.6953333,4.443587,15157,10216018,6720260
2022-03,0.8333333,6.93,3986.7,3.4,0.6756667,4.294002,37892,8925030,5644293
2022-06,1.5833333,7.30,3875.0,3.5,0.6510000,4.295036,221706,10670814,5800983


In [16]:
data_file_path <- paste(here(), "data", "merge_data.csv", sep = "/")
write_csv(all_data, data_file_path)

Toursim Data Merge (Come from two following files)
#tourism_final_clean_arriavals_1921Q1-2022Q2
#